In [2]:
import pandas as pd
df = pd.read_csv('../Data/lstm.csv')
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [3]:
df.category.unique()

array(['food', 'sports'], dtype=object)

> 중복된것을 제거하고 대소문자를 찢어놔야된다.


In [5]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
results = set() # 중복 제거 : set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size


536

In [6]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [7]:
from tensorflow import keras
# keras에서 setseed한 것
keras.utils.set_random_seed(1)

In [9]:
# One hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
encoded_paragraphs

[[262,
  368,
  513,
  434,
  11,
  17,
  38,
  368,
  305,
  50,
  245,
  447,
  13,
  434,
  83,
  26,
  236,
  120,
  161,
  168,
  320,
  270,
  285,
  329,
  466,
  519,
  324,
  121,
  120,
  273,
  29,
  103,
  148,
  456,
  268,
  27,
  294,
  237,
  273,
  173,
  394,
  324,
  232,
  179,
  45,
  438,
  345,
  351,
  436,
  245,
  84,
  120,
  478],
 [374,
  83,
  26,
  407,
  168,
  173,
  285,
  13,
  333,
  49,
  95,
  74,
  519,
  385,
  203,
  170,
  175,
  325,
  70],
 [298,
  479,
  421,
  4,
  357,
  12,
  519,
  380,
  187,
  58,
  479,
  273,
  40,
  380,
  357,
  12,
  368,
  184,
  71,
  486,
  274,
  96,
  314,
  8,
  251,
  175,
  29,
  120,
  502,
  388,
  351,
  479,
  308,
  519,
  350,
  384,
  120,
  478,
  71,
  200,
  103,
  268,
  3],
 [373,
  142,
  40,
  212,
  87,
  245,
  465,
  519,
  8,
  500,
  519,
  465,
  390,
  245,
  217,
  237,
  90,
  519,
  245,
  390,
  237,
  40,
  169,
  183,
  54,
  371,
  245,
  390,
  367,
  436,
  284,
  369,
  13,
 

In [11]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인
max_length = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)    

91


In [13]:
# 문장마다 단어 갯수가 다르므로 sequencce padding을 넣어서 문장의 길이를 동일하게 만듭니다.
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(
    encoded_paragraphs, maxlen = max_length, padding='post'
)
padded_paragraphs_encoding


array([[262, 368, 513, ...,   0,   0,   0],
       [374,  83,  26, ...,   0,   0,   0],
       [298, 479, 421, ...,   0,   0,   0],
       ...,
       [132, 363, 103, ...,   0,   0,   0],
       [ 13, 245, 426, ...,   0,   0,   0],
       [421, 121, 245, ...,   0,   0,   0]], dtype=int32)

In [14]:
# 분류항목(food, sports)를 수치로 변경하기

categories = df['category'].to_list()

def category_encode(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]

In [15]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [16]:
# Feature 확인
print(encoded_paragraphs[0])

[262, 368, 513, 434, 11, 17, 38, 368, 305, 50, 245, 447, 13, 434, 83, 26, 236, 120, 161, 168, 320, 270, 285, 329, 466, 519, 324, 121, 120, 273, 29, 103, 148, 456, 268, 27, 294, 237, 273, 173, 394, 324, 232, 179, 45, 438, 345, 351, 436, 245, 84, 120, 478]


In [17]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [26]:
model = keras.Sequential()
# 문맥 생성 단계
model.add(keras.layers.Embedding(vocab_size, 5, input_length=max_length))
# embedding 레이어는 인덱스를 받아 5차원 벡터의 embedding을 출력합니다. 
# 5차원인 이유는 과거, 과거진행, 과거완료, 현재, 현재진행, 현재완료 ? 아님 5형식?
model.add(keras.layers.LSTM(64))
# 분류단계
model.add(keras.layers.Dense(32, activation='relu')) # LSTM 반값
model.add(keras.layers.Dense(2, activation='softmax')) # 문장은 확률값을 구해야하므로

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [28]:
import numpy as np
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [29]:
# 데이터가 많으면 batch_size를 써줘야된다.
model.fit(train_X, train_Y, batch_size=10, epochs=50)

Epoch 1/50
2/2 [==============================] - 2s 21ms/step - loss: 0.6938 - accuracy: 0.4000
Epoch 2/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6928 - accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 30ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 29ms/step - loss: 0.6922 - accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6920 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 8/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6909 - accuracy: 0.5000
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6903 - accuracy: 0.5000
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6897 - accuracy: 0.6500
Epoch 11/50
2/2 [============